In [1]:
import numpy as np
import cv2
from collections import deque
from keras.models import model_from_json
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
model = model_from_json(open('model.json','r').read())
model.load_weights('weights.h5')

In [3]:
letter_count = {0: 'CHECK', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j',
                    11: 'k',
                    12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v',
                    23: 'w',
                    24: 'x', 25: 'y', 26: 'z', 27: 'CHECK'}


In [4]:
def process_image(img):
    image_x = 28
    image_y = 28
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [5]:
def predict_label(model, img):
    processed = process_image(img)
#     print("processed: " + str(processed.shape))
    pred_prob = model.predict(processed)[0]
    pred_class = list(pred_prob).index(max(pred_prob))
    return max(pred_prob), pred_class

In [8]:
cap = cv2.VideoCapture(0)
Lower_green = np.array([110,50,50])
Upper_green = np.array([130,255,255])

pred_class = 0

pts = deque(maxlen=512)
blackboard = np.zeros((480,640,3), dtype=np.uint8)
digit = np.zeros((200,200,3), dtype=np.uint8)

while True:
    ret, img = cap.read()
    img = cv2.flip(img,1)
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv_img, Lower_green, Upper_green)
    blur = cv2.medianBlur(mask, 15)
    blur = cv2.GaussianBlur(blur, (5,5),0)
    thresh = cv2.threshold(blur, 0,255,cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
    center = None
    
    if len(cnts) >=1:
        contour = max(cnts, key=cv2.contourArea)
        if cv2.contourArea(contour) > 250:
            ((x,y), radius) = cv2.minEnclosingCircle(contour)
            cv2.circle(img, (int(x), int(y)), int(radius), (0,255,255), 2)
            cv2.circle(img, center, 5,(0,0,255), -1)
            
            M = cv2.moments(contour)
            center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
            pts.appendleft(center)
            
            for i in range (1, len(pts)):
                if pts[i-1] is None or pts[i] is None:
                    continue
                cv2.line(blackboard, pts[i-1], pts[i], (255,255,255), 10)
                cv2.line(img, pts[i-1], pts[i], (0,0,255),5)
    elif len(cnts) ==0:
        if len(pts) !=[]:
            blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
            blur1 = cv2.medianBlur(blackboard_gray, 15)
            blur1 = cv2.GaussianBlur(blur1, (5,5), 0)
            thresh1 = cv2.threshold(blur1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
            blackboard_cnts = cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
            
            if len(blackboard_cnts) >=1:
                cnt = max(blackboard_cnts, key=cv2.contourArea)
#                 print(cv2.contourArea(cnt))
                
                if cv2.contourArea(cnt) > 2000:
                    x,y,w,h = cv2.boundingRect(cnt)
                    digit = blackboard_gray[y:y+h, x:x+w].T
                    pred_prob, pred_class = predict_label(model, digit)
                    print(pred_class, pred_prob)
                    
        pts = deque(maxlen=512)
        blackboard = np.zeros((480,640,3), dtype = np.uint8)
    
    cv2.putText(img, 'Conv Network : ' + str(letter_count[pred_class]), (10,470),
               cv2.FONT_HERSHEY_COMPLEX, 0.7, (0,0,255), 2)
    
    cv2.imshow('Frame', img)
    cv2.imshow('blackboard', blackboard)
    cv2.imshow('Contours', thresh1)
    k=cv2.waitKey(10)
    if k == 27:
        break
        
cap.release()
cv2.destroyAllWindows()
            
            
            
            
            

1 1.0
4 0.9988858
24 1.0
1 0.933176
3 0.99999774
17 1.0
14 0.9999999
18 1.0
25 1.0
9 0.779411
10 0.99522865
18 1.0
5 1.0
22 1.0
6 1.0
4 0.94057715
16 0.99988425
